# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation')
print(sys.path)

In [ ]:
import re
import swifter
import pandas as pd
import config as cfg
from pshmodule.utils import filemanager as fm

원본

In [ ]:
df_origin = fm.load(cfg.temp_ref_data_colab)

In [ ]:
df_origin.head()

,u,cpr,mm,cpo
0,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 것도 곁들인...,NaN
1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 몫을 제일 많이 곁들인...,NaN
2,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 대춘이 몫도 곁들인...,NaN
3,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,NaN
4,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 과소비를 곁들인...,NaN


arg 합치기

In [ ]:
df1 = fm.load(cfg.data_path_v1)

extension : .xlsx
Loaded 36602 records from drive/MyDrive/MemeProject//data/row_data/meme_v1.xlsx


In [ ]:
df2 = fm.load(cfg.data_path_v2)

extension : .xlsx
Loaded 14640 records from drive/MyDrive/MemeProject//data/row_data/meme_v2.xlsx


In [ ]:
new_header = df1.iloc[1]
df1 = df1[2:]
df1.columns = new_header

In [ ]:
df_ref = pd.concat([df1, df2])

In [ ]:
df_ref.reset_index(inplace=True)

In [ ]:
df_ref = df_ref[['발화문', 'ARG1', 'ARG_op']]

In [ ]:
df_ref.rename(columns={'발화구분':'type', '발화문':'u', 'ARG1':'arg1', 'ARG_op':'arg_op'}, inplace=True)

In [ ]:
df_ref.head()

,u,arg1,arg_op
0,나 중간고사 반에서 1등했어,중간고사,반에서
1,중간고사 점수 내가 반에서 제일 잘 받음,중간고사,반에서
2,나 반에서 중간 성적 제일 좋아,중간,반에서
3,우리 반에서 내가 시험 제일 잘 봤다,시험,우리 반에서
4,중간고사 반 1등 먹음,중간고사,반


In [ ]:
df_ref.shape

(51240, 3)

중복 제거

In [ ]:
result_u = ['']
result_arg1 = ['']
result_arg_op = ['']

for i in df_ref.iterrows():
  if i[1]['u'] not in result_u:
    result_u.append(i[1]['u'])
    result_arg1.append(i[1]['arg1'])
    result_arg_op.append(i[1]['arg_op'])

In [ ]:
print(len(result_u))
print(len(result_arg1))
print(len(result_arg_op))

22653
22653
22653


6개씩 조합하기

In [ ]:
df_ref = pd.DataFrame({'u':result_u, 'arg1':result_arg1, 'arg_op':result_arg_op})

In [ ]:
df_ref.head()

,u,arg1,arg_op
0,,,
1,나 중간고사 반에서 1등했어,중간고사,반에서
2,중간고사 점수 내가 반에서 제일 잘 받음,중간고사,반에서
3,나 반에서 중간 성적 제일 좋아,중간,반에서
4,우리 반에서 내가 시험 제일 잘 봤다,시험,우리 반에서


In [ ]:
df_ref = df_ref[1:]

In [ ]:
df_ref.reset_index(inplace=True, drop=True)

In [ ]:
df_ref.head()

,u,arg1,arg_op
0,나 중간고사 반에서 1등했어,중간고사,반에서
1,중간고사 점수 내가 반에서 제일 잘 받음,중간고사,반에서
2,나 반에서 중간 성적 제일 좋아,중간,반에서
3,우리 반에서 내가 시험 제일 잘 봤다,시험,우리 반에서
4,중간고사 반 1등 먹음,중간고사,반


합치기

In [ ]:
df = pd.merge(left = df_origin , right = df_ref, how = "inner", on = "u")

In [ ]:
df = df.fillna('')

In [ ]:
df.head()

,u,cpr,mm,cpo,arg1,arg_op
0,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 것도 곁들인...,,중간고사,반에서
1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 내 몫을 제일 많이 곁들인...,,중간고사,반에서
2,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 대춘이 몫도 곁들인...,,중간고사,반에서
3,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,,중간고사,반에서
4,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지!,근데 이제 과소비를 곁들인...,,중간고사,반에서


대답 합치기

In [ ]:
answer = [str(i[1]['cpr']) +" "+ str(i[1]['mm']) + " " + str(i[1]['cpo']) for i in df.iterrows()]

In [ ]:
df['answer'] = answer

In [ ]:
df = df[['u', 'answer', 'arg1', 'arg_op']]

In [ ]:
df.head()

,u,answer,arg1,arg_op
0,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,반에서
1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 몫을 제일 많이 곁들인...,중간고사,반에서
2,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 대춘이 몫도 곁들인...,중간고사,반에서
3,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,중간고사,반에서
4,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 과소비를 곁들인...,중간고사,반에서


In [ ]:
num = []
count = 1

for i in range(len(df)):
  if i % 36 == 0 and i != 0:
    count += 1  
  num.append(count)

In [ ]:
print(df.shape)
print(len(num))

(208700, 5)
208700


In [ ]:
df['num'] = num

In [ ]:
df.head()

,num,u,answer,arg1,arg_op
0,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,반에서
1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 몫을 제일 많이 곁들인...,중간고사,반에서
2,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 대춘이 몫도 곁들인...,중간고사,반에서
3,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,중간고사,반에서
4,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 과소비를 곁들인...,중간고사,반에서


In [ ]:
df = df[['num', 'u', 'answer', 'arg1', 'arg_op']]

temp save

In [ ]:
fm.save(cfg.aug_temp, df)

Saved 208700 records


# Replace & Augmentation

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/doc2vec/data_augmentation']


In [2]:
from tqdm import tqdm
import config as cfg
from pshmodule.utils import filemanager as fm

In [3]:
df = fm.load(cfg.aug_temp)

extension : .pickle
Loaded 208700 records from drive/MyDrive/MemeProject/data/augmentation/aug_temp.pickle


In [4]:
df.tail()

,num,u,answer,arg1,arg_op
208695,5798,전국노래자랑에서 이모가 1등했다,대길이 이모님... 힘숨찐ㄷㄷ 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서
208696,5798,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시네! 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서
208697,5798,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시라능 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서
208698,5798,전국노래자랑에서 이모가 1등했다,대길좌 이모님이 우승자라구?! 상상도 못한 정체 ㄴㅇㄱ,,전국노래자랑에서
208699,5798,전국노래자랑에서 이모가 1등했다,"대길쓰 이모님,, 힘숨찐ㄷㄷ 상상도 못한 정체 ㄴㅇㄱ",,전국노래자랑에서


In [5]:
check = df[df['u']=='얘 밥통이네']

In [6]:
check.head()

,num,u,answer,arg1,arg_op
366,11,얘 밥통이네,이게 맞아? 아니라고 왜 말을 못해!,,
367,11,얘 밥통이네,이게 맞아? 나는 아닌 것 같은디... 기분 탓인감,,
368,11,얘 밥통이네,이게 맞아? 대춘이도 상처 받을 줄 알거든?,,
369,11,얘 밥통이네,이게 맞아? 확실해???,,
370,11,얘 밥통이네,이게 맞아? 대길이 너무해~,,


216 마다 처리


In [14]:
num = []
u = []
answer = []
arg1 = []
arg_op = []
result = []


for i in tqdm(df.iterrows()):
  num.append(i[1]['num'])
  u.append(i[1]['u'])
  answer.append(i[1]['answer'])
  arg1.append(i[1]['arg1'])
  arg_op.append(i[1]['arg_op'])

  if i[0] % 215 == 0 and i[0] != 0:
    # 문장 6개씩 조합
    set_arg1 = set(arg1)
    set_argop = set(arg_op)
    set_arg1.discard('')
    set_argop.discard('')

    for z in zip(num, u, answer):
      # arg1 교체
      arg1_split = z[1].split()
      for k, s in enumerate(arg1_split):
        if not set_arg1:
          result.append([z])
        elif s in set_arg1:
          for w in set_arg1:
            arg1_split[k] = w
            z1 = ' '.join(arg1_split)

            # arg_op 교체
            argop_split = z1.split()
            for kk, ss in enumerate(argop_split):
              if not set_argop:
                result.append([z[0], z1, z[2]])
              elif ss in set_argop:
                for ww in set_argop:
                  argop_split[kk] = ww
                  z2 = ' '.join(argop_split)
                  result.append([z[0], z2, z[2]])          

    # init
    num.clear()
    u.clear()
    answer.clear()
    arg1.clear()
    arg_op.clear()

208700it [00:18, 11030.37it/s]


In [17]:
print(df.shape)
print(len(result))

(208700, 5)
1282547


In [ ]:
# sentence = ['나 중간고사 반에서 1등했어',
#              '중간고사 점수 내가 반에서 제일 잘 받음',
#              '나 반에서 중간 성적 제일 좋아',
#              '우리 반에서 내가 시험 제일 잘 봤다',
#              '중간고사 반 1등 먹음',
#              '중간시험 내가 반 1등이야']

In [ ]:
# arg1 = ['중간고사', '중간', '시험', '중간시험']
# arg2 = ['반에서', '우리 반에서', '반']

In [ ]:
# # 명사 교체
# result = []
# for s in sentence:
#   # 원본
#   if s not in result:
#     result.append(s)
#   temp = s.split()
#   output = ""
#   # arg1
#   for k, v in enumerate(temp):
#     if v in arg1:
#       for a1 in arg1:
#         temp[k] = a1
#         output = ' '.join(temp)
#         if output not in result:
#           result.append(output)

In [ ]:
# # 조사 교체
# result2 = [] 
# for r in result:
#   # 원본
#   if r not in result:
#     result2.append(r)
#   temp2 = r.split()
#   # arg2
#   for k, v in enumerate(temp2):
#     if v in arg2:
#       for a2 in arg2:
#         temp2[k] = a2
#         output2 = ' '.join(temp2)
#         if output2 not in result2:
#           result2.append(output2)

In [ ]:
# pprint(result2)